## 連結mongoDB

In [7]:
import pymongo  #version:3.4.0
from pymongo import MongoClient
client = pymongo.MongoClient()
# from pymongo import Connection
# connection = Connection()#這是在自己的本機建的，所以用connection

In [8]:
db = client.Candidate
db.collection_names()

[u'comments', u'posts', u'pure_ke_user_copy', u'detector_one_by_one', u'user']

## 連結sqlite 

In [1]:
import sqlite3
conn = sqlite3.connect('candidate')
con = conn.cursor()

In [ ]:
## 刪除一個資料表
con.execute('''DROP TABLE clean_tsai_comment''')

In [2]:
# Create table
con.execute('''CREATE TABLE clean_tsai_comment
             (user_id text, 
             user_name text, 
             comment_id text PRIMARY KEY, 
             comment text, 
             comment_time date,
             page_name text, 
             post_id text)''')

In [3]:
facebook_id = '46251501064' #Tsai
facebook_page_name = 'Tsai'
# facebook_id = '136845026417486' #Ke
# facebook_page_name = 'Ke'
# facebook_id = '188311137478' #ChinaTimes 
# facebook_page_name = 'ChinaTimes'
# facebook_id = '241284961029'
# facebook_page_name = 'udn' 
# facebook_id = '394896373929368' 
# facebook_page_name = 'ltn' 
# facebook_id = '232633627068' 
# facebook_page_name = 'AppleDaily'

In [4]:
# 匯入在2015-2017年間留言多於一則以上的使用者名單
user_list = []
file = open('user_Tsai.txt', 'r') 
userfile = file.read()
# print userfile.split(',')[0]
user_list.append(userfile.split(','))
print user_list[0][0:20]
print len(user_list[0])

['1307550719306500', '282520725500399', '402184603467122', '1612531879075926', '357649311284175', '1077495905651938', '879672328842272', '361800747535726', '10206215950148794', '360637757653227', '167386673755045', '10211808906327222', '2229294480629446', '1747490062236139', '1223024557715780', '1151289914941098', '1419903194695476', '1423058981047363', '243217906134585', '267227073641351']
43725


## 寫進資料庫中

In [ ]:
# 寫進sqlite中
for u in user_list[0]:
    try:
        t = (u,)
        if len(con.execute('SELECT * FROM comments WHERE user_id=?',t,).fetchall())==0:  
            row = [ c['from']['id'],
                c['from']['name'],
                c['id'],
                c['message'],
                str(t[1])+'/'+str(t[2])+'/'+str(y)+' '+str(t[3])+':'+str(t[4]),
               facebook_page_name,
               c['id'].split('_')[0]]

            try:
                con.execute("INSERT INTO clean_tsai_comment (user_id, user_name, comment_id, comment, comment_time,page_name, post_id) VALUES (?,?,?,?,?,?,?)",row)
            except sqlite3.IntegrityError:
                print('ERROR: ID already exists in PRIMARY KEY column {}'.format(c['from']['id']))    

        

In [11]:
#按照user去抓comment標記,並以user為key存起來（存在叫做db.clean_comment中）
# 記得要改comment的名字
for u in user_list[0]:
    try:
        for c in db.comments.find({'user_id':u,'page_name':'Tsai','year':{'$gte':2015,'$lte':2017}}):
            if db.tsai_clean_comment.find({'user_id':c['user_id']}).count()==0:
                db.tsai_clean_comment.insert({'user_id':c['user_id'],
                              'user_name':c['user_name'],
                                'tsai_comment':[[c['comment']]],
                                'tsai_comment_id':[[c['comment_id']]],
                                'sup_page_name':facebook_page_name,
                                'sup_page_id':facebook_id,
                                'n_sup_page_name':'',
                                'n_sup_page_id':'',
                                'n_page_name':'',
                                'n_page_id':'',
                               })
            else:
                db.test_tsai_clean_comment.update_one({'user_id':c['user_id'].encode('utf-8')},
                               {'$push':{
                                'tsai_comment_id':[c['comment_id']],
                                'tsai_comment':[c['comment']]}})
    except:
        print u
        pass
db.tsai_clean_comment.count()
db.update
# comments.update()
print db.tsai_clean_comment.count()
client.close()
# db.posts.find({})


/Users/TingYen/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:16: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.


0
